In [4]:
# Client Setup
import boto3

client = boto3.client("bedrock-runtime", region_name="us-east-2")
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [ ]:
# Helper functions


def add_user_message(messages, text):
    user_message = {"role": "user", "content": [{"text": text}]}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {
        "role": "assistant",
        "content": [{"text": text}],
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

In [6]:
# Tools and Schemas

from datetime import datetime, timedelta


def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29
                if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
                else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(
        f"----\nSetting the following reminder for {timestamp}:\n{content}\n----"
    )


get_current_datetime_schema = {
    "toolSpec": {
        "name": "get_current_datetime",
        "description": "Retrieves the current date and time formatted according to the specified format string. This tool returns the system's current local date and time as a formatted string. It should be used when the user needs to know the current date, time, or both in a specific format. The tool uses Python's datetime.now() function internally, so it will reflect the server's local time zone settings. This tool does not support retrieving dates or times for different time zones or historical/future dates.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "date_format": {
                        "type": "string",
                        "description": "A string that specifies the format of the returned date and time, using Python's strftime format codes. For example, '%Y-%m-%d' returns only the date as '2025-04-02', '%H:%M:%S' returns only the time as '14:30:45', and the default '%Y-%m-%d %H:%M:%S' returns both as '2025-04-02 14:30:45'. Use format codes like %Y (4-digit year), %m (month), %d (day), %H (hour), %M (minute), and %S (second) to customize the output format.",
                    }
                },
            },
        },
    }
}


add_duration_to_datetime_schema = {
    "toolSpec": {
        "name": "add_duration_to_datetime",
        "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "datetime_str": {
                        "type": "string",
                        "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
                    },
                    "duration": {
                        "type": "number",
                        "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
                    },
                    "input_format": {
                        "type": "string",
                        "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
                    },
                },
                "required": ["datetime_str"],
            }
        },
    }
}

set_reminder_schema = {
    "toolSpec": {
        "name": "set_reminder",
        "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
                    },
                    "timestamp": {
                        "type": "string",
                        "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
                    },
                },
                "required": ["content", "timestamp"],
            }
        },
    }
}

batch_tool_schema = {
    "toolSpec": {
        "name": "batch_tool",
        "description": "Invoke multiple other tool calls simultaneously",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "invocations": {
                        "type": "array",
                        "description": "The tool calls to invoke",
                        "items": {
                            "types": "object",
                            "properties": {
                                "name": {
                                    "types": "string",
                                    "description": "The name of the tool to invoke",
                                },
                                "arguments": {
                                    "types": "string",
                                    "description": "The arguments to the tool, encoded as a JSON string",
                                },
                            },
                            "required": ["name", "arguments"],
                        },
                    }
                },
                "required": ["invocations"],
            }
        },
    }
}


## 1. Tool Functions

Best Practices:

- Use well-named, descriptive arguments (important for later)
- Validate the inputs, raising errors if they fail
- Return meaningful errors - Claude will try to call your function a second time if it get's an error

### Get Current Date & Time

In [28]:
def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    return datetime.now().strftime(date_format)

get_current_datetime()

'2025-09-07 22:30:56'

## 2. JSON Schema Specifications

![JSON Schema Spec](https://everpath-course-content.s3-accelerate.amazonaws.com/instructor%2Fa46l9irobhg0f5webscixp0bs%2Fpublic%2F1748558031%2F08_-_003_-_JSON_Schema_for_Tools_04.1748558031698.png)

Purpose:
1. Helps LLMs understand what arguments are required by the tool function.
2. Used for data validation across many programming contexts.


Best Practices:
- Explain what the tool does, when to use it and what it returns
- Aim for 3 to 4 sentences in your descriptions
- Provide detailed descriptions for parameters

In [1]:
# 1. send this to the JSON to JSON Schema Converter
current_date_time_json = {
    "date_format": "%Y-%m-%d %H:%M:%S"
}

In [2]:
# 2. adding the description of the parameters 
current_date_time_json_schema = {
  "type": "object",
  "properties": {
    "date_format": {
      "type": "string",
      "description": "The format for returning the date-time as per Python strftime."
    }
  },
  "required": [
    "date_format"
  ]
}

In [3]:
# 3. putting it all together
current_date_time_json_schema = {
    "toolSpec": {
        "name": "get_current_date_time",
        "description": "Retrieves the current date and time formatted according to the specified format string. This tool returns the system's current local date and time as a formatted string. It should be used when the user needs to know the current date, time, or both in a specific format. The tool uses Python's datetime.now() function internally, so it will reflect the server's local time zone settings. This tool does not support retrieving dates or times for different time zones or historical/future dates.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "date_format": {
                        "type": "string",
                        "description": "The format for returning the date-time as per Python strftime."
                    }
                },
                "required": [
                    "date_format"
                ]
            }
        }
    } 
}

## 3. Call Claude with JSON Schema

In [8]:
def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    if tools:
        params["toolConfig"] = {"tools": tools, "toolChoice": {"auto": {}}}
        # toolChoice 
            # - auto is the default so technically we don't need to mention it.
            # - other options:
                # - "any": {}
                # - "tool": {"name": "tool-name"}

    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

### Controlling Tool Use Using 'toolChoice'
![auto, any and tool](https://everpath-course-content.s3-accelerate.amazonaws.com/instructor%2Fa46l9irobhg0f5webscixp0bs%2Fpublic%2F1748558036%2F08_-_004_-_Handling_Tool_Use_Responses_02.1748558036403.png)

In [ ]:
# without tool call

messages = []

add_user_message(messages, "What time is it?")
chat(messages)

"I don't have the ability to determine the current time for your location as I don't have access to real-time data or your device's clock. To check the current time, you could look at your device's clock, ask a smart assistant like Siri or Google Assistant, or visit a time website."

In [10]:
# with tool call

messages = []

add_user_message(messages, "What time is it?")
chat(messages, tools=[get_current_datetime_schema])

'I can check the current time for you. Let me retrieve that information for you right now.'

Let's see what's going on by printing the response from the chat function.

In [11]:
def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    if tools:
        params["toolConfig"] = {"tools": tools, "toolChoice": {"auto": {}}}
        # toolChoice 
            # - auto is the default so technically we don't need to mention it.
            # - other options:
                # - "any": {}
                # - "tool": {"name": "tool-name"}

    response = client.converse(**params)
    print(response)

    return response["output"]["message"]["content"][0]["text"]

In [12]:
# with tool call

messages = []

add_user_message(messages, "What time is it?")
chat(messages, tools=[get_current_datetime_schema])

{'ResponseMetadata': {'RequestId': 'b572382e-a05d-4b7f-a72f-126e09bbab80', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 07 Sep 2025 17:11:58 GMT', 'content-type': 'application/json', 'content-length': '486', 'connection': 'keep-alive', 'x-amzn-requestid': 'b572382e-a05d-4b7f-a72f-126e09bbab80'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'I can help you find out the current time. Let me check that for you.'}, {'toolUse': {'toolUseId': 'tooluse_0-ZlzEmcRNaHuiNSnjVw7A', 'name': 'get_current_datetime', 'input': {'date_format': '%H:%M:%S'}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 614, 'outputTokens': 80, 'totalTokens': 694, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 1855}}


'I can help you find out the current time. Let me check that for you.'

In [14]:
def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    if tools:
        params["toolConfig"] = {"tools": tools, "toolChoice": {"auto": {}}}
        # toolChoice 
            # - auto is the default so technically we don't need to mention it.
            # - other options:
                # - "any": {}
                # - "tool": {"name": "tool-name"}

    response = client.converse(**params)
    
    text = response["output"]["message"]["content"][0]["text"]
    parts = response["output"]["message"]["content"]

    # parts looks like this:
    # "content": [
    #             {"text": "I can help you find out the current time. Let me check that for you."}, 
    #             {"toolUse": {
    #                 "toolUseId": "tooluse_0-ZlzEmcRNaHuiNSnjVw7A", 
    #                 "name": "get_current_datetime", 
    #                 "input": {"date_format": "%H:%M:%S"}
    #             }}
    #         ]

    return text, parts

The chat function above now gives us both the text and some parts (toolUse). However we need to add this the tool use information into the user message and assistant messages:

In [15]:
def add_user_message(messages, content):
    # if the content is a string
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    # if it is a dictionary
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": content,
        }
    messages.append(assistant_message)

In [16]:
# with tool call

messages = []

add_user_message(messages, "What time is it?")
text, parts = chat(messages, tools=[get_current_datetime_schema])

In [17]:
add_assistant_message(messages, parts)

messages

[{'role': 'user', 'content': [{'text': 'What time is it?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll check the current time for you."},
   {'toolUse': {'toolUseId': 'tooluse_O0WgCE_6TWuZIoxPTLwxpw',
     'name': 'get_current_datetime',
     'input': {'date_format': '%Y-%m-%d %H:%M:%S'}}}]}]

Sometimes, Claude might decide that it wants to run multiple tools, so it will give us multiple toolUse dictionaries in the contents. So when we want to run the tools, we should make sure to run all the toolUse parts.

# 4. Run Tool & Send Tool Result Part

## Run Tool:

In [29]:
# runs the specific tool based on the input it requires
def run_tool(tool_name, tool_input):
    # TODO: tutorial says to hard-code but there must be a better way
    if tool_name == "get_current_datetime":
        # we use the ** in the input because it is a dictionary input
        return get_current_datetime(**tool_input)
    else:
        raise Exception(f"Unknown tool name: {tool_name}")

In [30]:
# this is the function that goes thru each of the tools and calls the run_tool function
def run_tools(parts):
    tool_requests = [part for part in parts if "toolUse" in part]
    tool_result_parts = []

    for tool_request in tool_requests:
        tool_use_id = tool_request["toolUse"]["toolUseId"]
        tool_name = tool_request["toolUse"]["name"]
        tool_input = tool_request["toolUse"]["input"]

        # print(f"I need to run tool {tool_name} with args {tool_input}")

        tool_output = run_tool(tool_name, tool_input)
        print(tool_output)

run_tools(parts)

2025-09-07 22:31:10


## Tool Result Part

Looks just like the tools use part:

![tool result structure](./../../images/ToolResultPart.png)

In [ ]:
import json

# runs the specific tool based on the input it requires
def run_tool(tool_name, tool_input):
    # TODO: tutorial says to hard-code but there must be a better way
    if tool_name == "get_current_datetime":
        # we use the ** in the input because it is a dictionary input
        return get_current_datetime(**tool_input)
    else:
        raise Exception(f"Unknown tool name: {tool_name}")

# this is the function that goes thru each of the tools and calls the run_tool function
def run_tools(parts):
    tool_requests = [part for part in parts if "toolUse" in part]
    tool_result_parts = []

    for tool_request in tool_requests:
        tool_use_id = tool_request["toolUse"]["toolUseId"]
        tool_name = tool_request["toolUse"]["name"]
        tool_input = tool_request["toolUse"]["input"]

        # print(f"I need to run tool {tool_name} with args {tool_input}")

        try:
            tool_output = run_tool(tool_name, tool_input)
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    # we use the json.dumps to stringify the output if it was a dictionary
                    "content": [{"text": json.dumps(tool_output)}], 
                    "status": "success"
                }
            }
        except Exception as e:
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    # we use the json.dumps to stringify the output if it was a dictionary
                    "content": [{"text": f"Error: {e}"}],
                    "status": "error"
                }
            }
        
        tool_result_parts.append(tool_result_part)
    
    return tool_result_parts

run_tools(parts)

[{'toolResult': {'toolUseId': 'tooluse_O0WgCE_6TWuZIoxPTLwxpw',
   'content': [{'text': '"2025-09-07 22:53:03"'}],
   'status': 'success'}}]

# 5. Send Tool Result to Claude

## Add Tool Result Into Messages List

In [34]:
messages = []

add_user_message(messages, "What time is it?")
text, parts = chat(messages, tools=[get_current_datetime_schema])

add_assistant_message(messages, parts)

In [35]:
import json

# runs the specific tool based on the input it requires
def run_tool(tool_name, tool_input):
    # TODO: tutorial says to hard-code but there must be a better way
    if tool_name == "get_current_datetime":
        # we use the ** in the input because it is a dictionary input
        return get_current_datetime(**tool_input)
    else:
        raise Exception(f"Unknown tool name: {tool_name}")

# this is the function that goes thru each of the tools and calls the run_tool function
def run_tools(parts):
    tool_requests = [part for part in parts if "toolUse" in part]
    tool_result_parts = []

    for tool_request in tool_requests:
        tool_use_id = tool_request["toolUse"]["toolUseId"]
        tool_name = tool_request["toolUse"]["name"]
        tool_input = tool_request["toolUse"]["input"]

        # print(f"I need to run tool {tool_name} with args {tool_input}")

        try:
            tool_output = run_tool(tool_name, tool_input)
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    # we use the json.dumps to stringify the output if it was a dictionary
                    "content": [{"text": json.dumps(tool_output)}], 
                    "status": "success"
                }
            }
        except Exception as e:
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    # we use the json.dumps to stringify the output if it was a dictionary
                    "content": [{"text": f"Error: {e}"}],
                    "status": "error"
                }
            }
        
        tool_result_parts.append(tool_result_part)
    
    return tool_result_parts

In [36]:
messages

[{'role': 'user', 'content': [{'text': 'What time is it?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find out the current time. Let me check that for you."},
   {'toolUse': {'toolUseId': 'tooluse_bbod_JYMTKawCgY4QiJbaQ',
     'name': 'get_current_datetime',
     'input': {'date_format': '%H:%M:%S'}}}]}]

In [37]:
add_user_message(messages, run_tools(parts))

In the cell above, the `run_tools` method is called and the output is set to the 'contents' variable as a dictionary.

In [38]:
messages

[{'role': 'user', 'content': [{'text': 'What time is it?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find out the current time. Let me check that for you."},
   {'toolUse': {'toolUseId': 'tooluse_bbod_JYMTKawCgY4QiJbaQ',
     'name': 'get_current_datetime',
     'input': {'date_format': '%H:%M:%S'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_bbod_JYMTKawCgY4QiJbaQ',
     'content': [{'text': '"09:52:39"'}],
     'status': 'success'}}]}]

In [39]:
text, parts = chat(messages, tools=[get_current_datetime_schema])

text

'The current time is 09:52:39.'

# Multi-Turn Conversations With Tools

## Refactoring When 'stopReason' Is Not Tool Use

In the above case, it was right to use the tool and then the result of running the tool was set in the content of the final user message. In case the user asks a question, which doesn't need tool use, based on our above logic, it would still create a tool_use call and get an empty response. 



In [40]:
# helper methods

def add_user_message(messages, content):
    # if the content is a string
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    # if it is a dictionary
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": content,
        }
    messages.append(assistant_message)

def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    if tools:
        params["toolConfig"] = {"tools": tools, "toolChoice": {"auto": {}}}
        # toolChoice 
            # - auto is the default so technically we don't need to mention it.
            # - other options:
                # - "any": {}
                # - "tool": {"name": "tool-name"}

    response = client.converse(**params)
    
    parts = response["output"]["message"]["content"]

    # parts looks like this:
    # "content": [
    #             {"text": "I can help you find out the current time. Let me check that for you."}, 
    #             {"toolUse": {
    #                 "toolUseId": "tooluse_0-ZlzEmcRNaHuiNSnjVw7A", 
    #                 "name": "get_current_datetime", 
    #                 "input": {"date_format": "%H:%M:%S"}
    #             }}
    #         ]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        # combining all the texts with a new line separator
        "text": "\n".join([p["text"] for p in parts if "text" in p ])
    }

In [41]:
def run_conversation(messages):
    while True:

        result = chat(messages=messages, tools=[get_current_datetime_schema])

        add_assistant_message(messages, result["parts"])
        print(result["text"])

        if(result["stop_reason"] != "tool_use"):
            break

        tool_result_parts = run_tools(result["parts"])
        add_user_message(messages, tool_result_parts)
    
    return messages

### Testing With Tool Use

In [42]:
messages = []
add_user_message(messages, "What time is it?")
run_conversation(messages)

I can check the current time for you. Let me retrieve that information.
The current date and time is 2025-09-08 10:34:31 (September 8, 2025 at 10:34:31 AM) in the system's local time zone.


[{'role': 'user', 'content': [{'text': 'What time is it?'}]},
 {'role': 'assistant',
  'content': [{'text': 'I can check the current time for you. Let me retrieve that information.'},
   {'toolUse': {'toolUseId': 'tooluse_0WTo_DeCSCiWAZbAqyRMVQ',
     'name': 'get_current_datetime',
     'input': {'date_format': '%Y-%m-%d %H:%M:%S'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_0WTo_DeCSCiWAZbAqyRMVQ',
     'content': [{'text': '"2025-09-08 10:34:31"'}],
     'status': 'success'}}]},
 {'role': 'assistant',
  'content': [{'text': "The current date and time is 2025-09-08 10:34:31 (September 8, 2025 at 10:34:31 AM) in the system's local time zone."}]}]

### Testing Without Tool Use

In [43]:
messages = []
add_user_message(messages, "What is 1+1?")
run_conversation(messages)

I don't need any special tools to answer this basic math question for you.

1 + 1 = 2

This is a fundamental arithmetic operation that adds two values of 1 together to get 2.


[{'role': 'user', 'content': [{'text': 'What is 1+1?'}]},
 {'role': 'assistant',
  'content': [{'text': "I don't need any special tools to answer this basic math question for you.\n\n1 + 1 = 2\n\nThis is a fundamental arithmetic operation that adds two values of 1 together to get 2."}]}]

# Multiple Tools

In [48]:
def run_tool(tool_name, tool_input):
    # TODO: tutorial says to hard-code but there must be a better way
    if tool_name == "get_current_datetime":
        # we use the ** in the input because it is a dictionary input
        return get_current_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    else:
        raise Exception(f"Unknown tool name: {tool_name}")

In [49]:
def run_conversation(messages):
    while True:

        result = chat(messages=messages, tools=[
            get_current_datetime_schema,
            add_duration_to_datetime_schema,
            set_reminder_schema
            ]
        )

        add_assistant_message(messages, result["parts"])
        print(result["text"])

        if(result["stop_reason"] != "tool_use"):
            break

        tool_result_parts = run_tools(result["parts"])
        add_user_message(messages, tool_result_parts)
    
    return messages

In [50]:
messages = []
add_user_message(messages, "Set a reminder to shop from Dollarama in 5 days.")
run_conversation(messages)

I'll help you set a reminder to shop from Dollarama in 5 days. To do this, I need to:

1. Get the current date and time
2. Calculate the date 5 days from now
3. Set the reminder with the appropriate content and timestamp

Let me set this up for you:
Now I'll calculate the date 5 days from today:
Now I'll set the reminder:
----
Setting the following reminder for 2025-09-13T11:02:42:
Shop from Dollarama
----
Perfect! I've set a reminder for you to "Shop from Dollarama" on Saturday, September 13, 2025 at 11:02:42 AM. You'll receive a notification at that time to remind you.


[{'role': 'user',
  'content': [{'text': 'Set a reminder to shop from Dollarama in 5 days.'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you set a reminder to shop from Dollarama in 5 days. To do this, I need to:\n\n1. Get the current date and time\n2. Calculate the date 5 days from now\n3. Set the reminder with the appropriate content and timestamp\n\nLet me set this up for you:"},
   {'toolUse': {'toolUseId': 'tooluse_-4IgMgVdQ2evV8Z7V7b3Ig',
     'name': 'get_current_datetime',
     'input': {'date_format': '%Y-%m-%d %H:%M:%S'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_-4IgMgVdQ2evV8Z7V7b3Ig',
     'content': [{'text': '"2025-09-08 11:02:42"'}],
     'status': 'success'}}]},
 {'role': 'assistant',
  'content': [{'text': "Now I'll calculate the date 5 days from today:"},
   {'toolUse': {'toolUseId': 'tooluse_icI9ptJbT8uJ1W9wyKJNzg',
     'name': 'add_duration_to_datetime',
     'input': {'datetime_str': '2025-09-08 11:02:42',
      'du